# Chapter 3 Exercises

## Exercise 3.1: Hand calculations for the trapezoidal method

Hand computations:
    $$ \int_1^3 2x^3 dx \approx \frac{f(1)+f(3)}{2} + f(2) = \frac{2+54}{2} + 16 = 44 $$

In [15]:
# trapezoidal_test_func.py
from trapezoidal import trapezoidal

def test_trapezoidal():
    f = lambda x: 2*x**3
    x0, x1, n = 1, 3, 2
    
    # Comparer hand computations with system output
    I_hand = 44
    I_sys = trapezoidal(f, x0, x1, n)
    error = abs(I_hand - I_sys)
    tol = 1e-16
    assert error < tol
    
test_trapezoidal()

## Exercise 3.2: Hand calculations for the midpoint method

Hand computations:
    $$ \int_1^3 2x^3 dx \approx f\left(\frac32\right) + f\left(\frac52\right) = 2 \left(\frac32\right)^3 + 2 \left(\frac52\right)^3 = 38 $$

In [14]:
# midpoint_test_func.py
from midpoint import midpoint

def test_midpoint():
    f = lambda x: 2*x**3
    x0, x1, n = 1, 3, 2
    
    # Comparer hand computations with system output
    I_hand = 38
    I_sys = midpoint(f, x0, x1, n)
    error = abs(I_hand - I_sys)
    tol = 1e-16
    assert error < tol
    
test_midpoint()

## Exercise 3.3: Compute a simple integral

Hand computations for approximations of $\int_2^6 x(x-1)dx$.

\begin{align}
  n &= 2, I_t = \left(\frac{f(2)+f(6)}{2} + f(4)\right) \left(\frac{6-2}{2}\right)
  = \left(\frac{2(2-1)+6(6-1)}{2} + 4(4-1)\right) (2) = 56 \\
  n &= 2, I_m = \left( f(3) + f(5) \right) \left(\frac{6-2}{2}\right)
  = (3(3-1) + 5(5-1))(2) = 52
\end{align}

Not needed, misread question.

In [25]:
# integrate_parabola.py
from trapezoidal import trapezoidal
from midpoint import midpoint
import sympy as sp

def test_methods():
    f = lambda x: x*(x-1)
    x0, x1 = 2, 6

    # Exact integral by sympy
    x = sp.symbols('x')
    I_exact = sp.integrate(f(x), (x, x0, x1))
    I_exact = I_exact.evalf()

    # numerical integration
    nvec = [2, 100]
    print "  n  trapezoidal       error |   midpoint       error"
    for n in nvec:
        I_t = trapezoidal(f, x0, x1, n)
        e_t = abs(I_t - I_exact)
        I_m = midpoint(f, x0, x1, n)
        e_m = abs(I_t - I_exact)
        print("%3i   %10.6f  %10.6f | %10.6f  %10.6f" % (n, I_t, e_t, I_m, e_m))
        
test_methods()

  n  trapezoidal       error |   midpoint       error
  2    56.000000    2.666667 |  52.000000    2.666667
100    53.334400    0.001067 |  53.332800    0.001067


## Exercise 3.4: Hand-calculations with sine integrals

$\int_0^\pi \sin(x) dx = -\cos(\pi) -(-\cos(0)) = 2$

In [37]:
from trapezoidal import trapezoidal
from midpoint import midpoint
from math import pi, sin

def test_integrate_sine():
    f = lambda x: sin(x)
    x0, x1, n = 0, pi, int(1e2Exercise 3.5: Make test functions for the midpoint methodjjjjjj   )
    exact = 2
    numer_t = trapezoidal(f, x0, x1, n)
    numer_m = midpoint(f, x0, x1, n)
    err_t = abs(exact - numer_t)
    err_m = abs(exact - numer_m)
    tol = 1e-6
    assert err_t < tol, "err_t = %g" % (err_t)
    assert err_m < tol, "err_m = %g" % (err_m)

test_integrate_sine()

AssertionError: err_t = 0.000164496

It converges rather slow.  Large `n` and `tol` are needed to avoid `AssertionError`.

## Exercise 3.5: Make test functions for the midpoint method

In [38]:
from midpoint import midpoint

def test_midpoint_one_exact_result():
    """Compare one hand-computed result."""
    from math import exp
    v = lambda t: 3*(t**2)*exp(t**3)
    n = 2
    computed = midpoint(v, 0, 1, n)
    expected = 1.3817914596908085
    error = abs(expected - computed)
    tol = 1E-14
    success = error < tol
    msg = 'error=%g > tol=%g' % (error, tol)
    assert success, msg

def test_midpoint_linear():
    """Check that linear functions are integrated exactly."""
    f = lambda x: 6*x - 4
    F = lambda x: 3*x**2 - 4*x  # Anti-derivative
    a = 1.2; b = 4.4
    expected = F(b) - F(a)
    tol = 1E-14
    for n in 2, 20, 21:
        computed = midpoint(f, a, b, n)
        error = abs(expected - computed)
        success = error < tol
        msg = 'n=%d, err=%g' % (n, error)
        assert success, msg

def convergence_rates(f, F, a, b, num_experiments=14):
    from math import log
    from numpy import zeros
    expected = F(b) - F(a)
    n = zeros(num_experiments, dtype=int)
    E = zeros(num_experiments)
    r = zeros(num_experiments-1)
    for i in range(num_experiments):
        n[i] = 2**(i+1)
        computed = midpoint(f, a, b, n[i])
        E[i] = abs(expected - computed)
        if i > 0:
            r_im1 = log(E[i]/E[i-1])/log(float(n[i])/n[i-1])
            r[i-1] = float('%.2f' % r_im1) # Truncate to two decimals
    return r

def test_midpoint_conv_rate():
    """Check empirical convergence rates against the expected -2."""
    from math import exp
    v = lambda t: 3*(t**2)*exp(t**3)
    V = lambda t: exp(t**3)
    a = 1.1; b = 1.9
    r = convergence_rates(v, V, a, b, 14)
    print r
    tol = 0.01
    msg = str(r[-4:])  # show last 4 estimated rates
    assert (abs(r[-1]) - 2) < tol, msg

In [40]:
test_midpoint_one_exact_result()

In [41]:
test_midpoint_linear()

In [42]:
test_midpoint_conv_rate()

[-1.43 -1.81 -1.95 -1.99 -2.   -2.   -2.   -2.   -2.   -2.   -2.   -2.
 -2.  ]


## Exercise 3.6: Explore rounding errors with large numbers

In [43]:
def test_trapezoidal_linear2():
    """Check that linear functions are integrated exactly."""
    f = lambda x: 6*1e8*x - 4*1e6
    F = lambda x: 3*1e8*x**2 - 4*1e6*x  # Anti-derivative
    a = 1.2; b = 4.4
    expected = F(b) - F(a)
    tol = 1E-14
    for n in 2, 20, 21:
        computed = trapezoidal(f, a, b, n)
        error = abs(expected - computed)
        success = error < tol
        msg = 'n=%d, err=%g' % (n, error)
        assert success, msg
        
test_trapezoidal_linear2()

AssertionError: n=2, err=9.53674e-07

Use relative error instead of absolute error to make the test useful.

In [45]:
def test_trapezoidal_linear3():
    """Check that linear functions are integrated exactly."""
    f = lambda x: 6*1e8*x - 4*1e6
    F = lambda x: 3*1e8*x**2 - 4*1e6*x  # Anti-derivative
    a = 1.2; b = 4.4
    expected = F(b) - F(a)
    tol = 1E-14
    for n in 2, 20, 21:
        computed = trapezoidal(f, a, b, n)
        error = abs(expected - computed)/abs(expected)
        success = error < tol
        msg = 'n=%d, err=%g' % (n, error)
        assert success, msg
        
test_trapezoidal_linear3()

No change is needed for the convergence rate test due to the $\log(E_{i}/E_{i-1})$ in numerator is invariant under scalar multiplication.

In [50]:
def test_trapezoidal_conv_rate2():
    """Check empirical convergence rates against the expected 0."""
    f = lambda x: 6*1e8*x - 4*1e6
    F = lambda x: 3*1e8*x**2 - 4*1e6*x  # Anti-derivative
    a = 1.2; b = 4.4
    r = convergence_rates(f, F, a, b, 14)
    print r
    tol = 0.01
    msg = str(r[-4:])  # show last 4 estimated rates
    assert abs(r[-1]) < tol, msg

test_trapezoidal_conv_rate2()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
